<font size="5">Cross approximation in the TT format</font>

Using the torchtt.TT constructor, a TT decomposition of a given tensor can be obtained. However, in the cases where the entries of the tensor are computed using a given function, building full tensors becomes unfeasible. It is possible to 

Imports:

In [1]:
import torch as tn
try:
    import torchtt as tntt
except:
    print('Installing torchTT...')
    %pip install git+https://github.com/ion-g-ion/torchTT

We want to approximate the tensor $\mathsf{x}_{i_1...i_d}=\frac{1}{2+i_1+\cdots+i_d}$. Since the passed indices are integers of type torch.int64, casting is used.

In [2]:
func1 = lambda I: 1/(2+tn.sum(I+1,1).to(dtype=tn.float64))

Call the torchtt.interpolate.dmrg_cross() method.

In [3]:
N = [20]*4
x = tntt.interpolate.dmrg_cross(func1, N, eps = 1e-7)

Compute the full tensor and compare to the reference.

In [4]:
Is = tntt.meshgrid([tn.arange(0,n,dtype=tn.float64) for n in N])
x_ref = 1/(2+Is[0].full()+Is[1].full()+Is[2].full()+Is[3].full()+4)
print('Relative error ',tn.linalg.norm(x.full()-x_ref)/tn.linalg.norm(x_ref))

Relative error  tensor(3.6063e-09, dtype=torch.float64)


We consider the case $d=10$, $n_i=32$. the full tensor would contain $32^10$ entries.

In [5]:
N = [32]*10
x = tntt.interpolate.dmrg_cross(func1, N, eps = 1e-10, verbose=True)

Sweep 1: 
	LR supercore 1,2
		number evaluations 2048
		rank updated 2 -> 7, local error 1.064103e+06
	LR supercore 2,3
		number evaluations 14336
		rank updated 2 -> 9, local error 1.146021e+00
	LR supercore 3,4
		number evaluations 18432
		rank updated 2 -> 10, local error 1.083790e+00
	LR supercore 4,5
		number evaluations 20480
		rank updated 2 -> 11, local error 1.147148e+00
	LR supercore 5,6
		number evaluations 22528
		rank updated 2 -> 11, local error 1.080203e+00
	LR supercore 6,7
		number evaluations 22528
		rank updated 2 -> 11, local error 1.027338e+00
	LR supercore 7,8
		number evaluations 22528
		rank updated 2 -> 14, local error 1.049664e+00
	LR supercore 8,9
		number evaluations 28672
		rank updated 2 -> 13, local error 1.061307e+00
	LR supercore 9,10
		number evaluations 13312
		rank updated 2 -> 12, local error 1.024649e+00
	RL supercore 9,10
		number evaluations 13312
		rank updated 12 -> 12, local error 1.643264e-12
	RL supercore 8,9
		number evaluations 172032
		ra

The adaptive cross method used only a fraction of function calls from the original tensor.
Check some entries (full tensor cannot be computed this time) and show the rank and the storage requirements.

In [6]:
print(x[1,2,3,4,5,6,7,8,9,11], ' reference ', func1(tn.tensor([[1,2,3,4,5,6,7,8,9,11]])))
print(x[12,23,17,25,30,0,7,8,9,11], ' reference ', func1(tn.tensor([[12,23,17,25,30,0,7,8,9,11]])))
print(x)

tensor(0.0147, dtype=torch.float64)  reference  tensor([0.0147], dtype=torch.float64)
tensor(0.0065, dtype=torch.float64)  reference  tensor([0.0065], dtype=torch.float64)
TT with sizes and ranks:
N = [32, 32, 32, 32, 32, 32, 32, 32, 32, 32]
R = [1, 13, 14, 14, 14, 14, 13, 13, 13, 11, 1]

Device: cpu, dtype: torch.float64
#entries 46624 compression 4.141043064009864e-11

